In [ ]:
from numpy.linalg import norm
from pathlib import Path

from collections import OrderedDict
from scipy import interpolate
import numpy as np


In [ ]:
def SpinExpenseFunction():
    """Returns a function that takes a list of dimensionless spins and
    outputs a list of CPU-h cost per orbit for that spin"""
    # CPU-h per orbit cost of each system
    data = OrderedDict(
        [
            (-0.9, 124404 / 12.6689613971),
            (-0.8, 39434 / 13.242325264),
            (-0.6, 22003 / 14.4755972672),
            (0.0, 15000 / 18.5),  # fiducial point to help spline
            (0.6, 31825 / 22.6434438811),
            (0.8, 68245 / 24.0885526795),
            (0.9, 137532 / 24.8307413102),
            (0.95, 192026 / 25.1507192987),
        ]
    )
    f = interpolate.UnivariateSpline(list(data.keys()), list(data.values()), k=1, s=0)
    return f


def CostInZwickyDays(q, chiA, chiB, nOrbits):
    spinCost = SpinExpenseFunction()

    # Make many wild unlisted assumptions that are almost certainly not true.
    chiA_z_cost = 0.5 * spinCost(chiA[2])
    chiB_z_cost = 0.5 * spinCost(chiB[2])
    chiA_mag_cost = 0.25 * (spinCost(norm(chiA)) + spinCost(-norm(chiA)))
    chiB_mag_cost = 0.25 * (spinCost(norm(chiB)) + spinCost(-norm(chiB)))
    chiA_cost = 0.5 * (chiA_z_cost + chiA_mag_cost)
    chiB_cost = 0.5 * (chiB_z_cost + chiB_mag_cost)
    cpuh_per_orbit = 0.25 * (1.0 + q) * (chiA_cost + chiB_cost)
    zwicky_days_cost = cpuh_per_orbit * nOrbits / (48 * 24)
    return zwicky_days_cost

def parse_params_input_file(param_file):
    contents = param_file.read_text().splitlines()
    params = {
        "SpinA": None,
        "SpinB": None,
        "MassRatio": None,
        "Omega0": None,
        "adot0": None,
        "D0": None,
    }
    for line in contents:
        for key in params.keys():
            if key in line and "Spin" in key:
                # @SpinA = (0.0, 0.0, 0.0);
                values = line.split("=")[1].strip()
                values = values.strip("();").split(",")
                params[key] = [float(v) for v in values]
            elif key in line:
                # $MassRatio = 1.0;
                value = line.split("=")[1].strip().strip(";")
                params[key] = float(value)
    return params

def get_next_RunID(ID_str):
    # "01" -> "02", "001" -> "002", "0009"-> "0010"
    width = len(ID_str)
    next_id = int(ID_str) + 1
    return str(next_id).zfill(width)


def get_params_from_last_Ecc_folder(run_folder_path):
    ecc_folders = [
        f for f in run_folder_path.iterdir() if f.is_dir() and f.name.startswith("Ecc")
    ]
    # Find the folder with largest Ecc<number> value
    ecc_names = [f.name for f in ecc_folders]
    max_ecc_folder = max(ecc_names, key=lambda name: float(name[3:]))
    max_ecc_folder_path = run_folder_path / max_ecc_folder
    return max_ecc_folder_path

In [ ]:
folder_path_list = [
    Path("/resnick/groups/sxs/hchaudha/spec_runs/47_LISA_cat/q1_0_0_753_0_0_087_no_NegExp"),
    Path("/resnick/groups/sxs/hchaudha/spec_runs/47_LISA_cat/q1_0_0_n8_0_0_n8_no_NegExp"),
    Path("/resnick/groups/sxs/hchaudha/spec_runs/47_LISA_cat/q1_0_0_n8_0_0_8_no_NegExp"),
    Path("/resnick/groups/sxs/hchaudha/spec_runs/47_LISA_cat/q1_0_0_8_0_0_8_no_NegExp"),
    Path("/resnick/groups/sxs/hchaudha/spec_runs/47_LISA_cat/q2_0_0_0_0_0_0_no_NegExp"),
    Path("/resnick/groups/sxs/hchaudha/spec_runs/47_LISA_cat/q3_0_0_0_0_0_0_no_NegExp"),
    Path("/resnick/groups/sxs/hchaudha/spec_runs/47_LISA_cat/q4_0_0_0_0_0_0_no_NegExp"),
    Path("/resnick/groups/sxs/hchaudha/spec_runs/52_athena_spin/q105_0_0_1_0_0_1"),
    Path("/resnick/groups/sxs/hchaudha/spec_runs/52_athena_spin/q105_0_0_4_0_0_4"),
    Path("/resnick/groups/sxs/hchaudha/spec_runs/52_athena_spin/q105_0_0_n1_0_0_1"),
    Path("/resnick/groups/sxs/hchaudha/spec_runs/52_athena_spin/q105_0_0_n4_0_0_4"),
]

In [ ]:
num_orbits = 40 # Hardcoded
last_run_ID = "01"


for folder_path in folder_path_list:
    last_ecc_folder = get_params_from_last_Ecc_folder(folder_path)
    param_file = last_ecc_folder / "Ev/JoinedForEcc/NewParams.input"
    # print(f"Reading params from {param_file}")
    params = parse_params_input_file(param_file)
    q = params["MassRatio"]
    chiA = params["SpinA"]
    chiB = params["SpinB"]
    Omega0 = params["Omega0"]
    adot0 = params["adot0"]
    D0 = params["D0"]

    cost = CostInZwickyDays(q, chiA, chiB, num_orbits)

    # RunID=01 ZwickyDays=14 q=1.0 chiA=0.0,0.0,0.0 chiB=0.0,0.0,0.0 D0=20.7902832031 Omega0=0.009934021788 adot0=-8.8739366796e-06

    last_run_ID = get_next_RunID(last_run_ID)
    print(f"RunID={last_run_ID} ZwickyDays={int(cost)} q={q} chiA={chiA[0]},{chiA[1]},{chiA[2]} chiB={chiB[0]},{chiB[1]},{chiB[2]} D0={D0} Omega0={Omega0} adot0={adot0}")
